In [1]:
# 로컬에서 pt파일을 이용하여 비속어 판별 테스트 

!pip install transformers

In [2]:
%pwd

'c:\\Files\\20173231\\활동\\2022_디지털 스마트 부산_ 스마트물류항만\\미니프로젝트\\3차 미니프로젝트'

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import torch
from transformers import AutoTokenizer,AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print("device:",device)

device: cpu


In [4]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
model = torch.load("sware_model(train32, eval64, epochs10).pt", map_location=device)
print(model)

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(50135, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

In [20]:
!pip install nltk

     ---------------------------------------- 1.5/1.5 MB 13.6 MB/s eta 0:00:00


In [5]:
MODEL_NAME = "beomi/KcELECTRA-base" # hugging face 에 등록된 모델
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def sentence_predict(sent):
  model.eval() # 평가

  # 입력문장 토크나이징
  tokenized_sent = tokenizer(
      sent,
      return_tensors="pt",
      truncation=True, 
      add_special_tokens=True, 
      max_length=128
      )

  # 모델 위치 gpu이동
  tokenized_sent.to(device)

  # 예측
  with torch.no_grad():
    outputs = model(
        input_ids=tokenized_sent["input_ids"],
        attention_mask=tokenized_sent["attention_mask"],
        token_type_ids=tokenized_sent["token_type_ids"],
    )

  # 결과
  logits = outputs[0]   ## 마지막 노드에서 아무런 Activation Function을 거치지 않은 값을 Logit
  logits = logits.detach().cpu()
  result = logits.argmax(-1)
  if result == 0:
    result = ">> 악성ㅋ"
  elif result ==1:
    result= ">> 정상ㅇ"
  return result
  

while True:
  sentence = input("댓글을 입력 해주세요:")
  if sentence == "0":
      break
  print(sentence_predict(sentence))
  print("\n")

Downloading:   0%|          | 0.00/504 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/396k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/124 [00:00<?, ?B/s]

>> 정상ㅇ


>> 정상ㅇ


>> 정상ㅇ


>> 악성ㅋ


>> 정상ㅇ


>> 악성ㅋ


>> 악성ㅋ


